# 02_05_codegeneration.ipynb - Code generation with Numba and Cython

**Problem: perform the integral**

$$\int_0^\pi \left[ \sum_{k=1}^{\infty} \frac{\cos(k \sin \theta)}{k^2} \right] d\theta$$

In [ ]:
import math
import numpy as np
import scipy
import scipy.integrate as si
import numba

In [ ]:
def integrand(theta):
    return sum(math.cos(k * math.sin(theta))/k**2 for k in range(1, 200))

In [ ]:
%time si.quad(integrand, 0, math.pi)

In [ ]:
@numba.jit
def integrand_nj(theta):
    return sum(math.cos(k * math.sin(theta))/k**2 for k in range(1, 200))

In [ ]:
integrand_nj(1)

In [ ]:
@numba.jit
def integrand_nj(theta):
    mysum = 0
    
    for k in range(1, 200):
        mysum = mysum + math.cos(k * math.sin(theta)) / k**2
    
    return mysum

In [ ]:
integrand_nj(1)

In [ ]:
si.quad(integrand_nj, 0, math.pi)

In [ ]:
%timeit si.quad(integrand_nj, 0, math.pi)

In [ ]:
%load_ext cython

In [ ]:
%%cython

# import trigonometric functions from the standard C library
from libc.math cimport sin, cos

def integrand_cc(double theta):
    cdef double mysum = 0.0
    cdef int k
    
    for k in range(1, 200):
        mysum = mysum + cos(k * sin(theta)) / k**2
        
    return mysum

In [ ]:
integrand_cc(1)

In [ ]:
si.quad(integrand_cc, 0, math.pi)

In [ ]:
%timeit si.quad(integrand_cc, 0, math.pi)

In [ ]:
def jacobi(array):    
    updated = array.copy()
    
    for i in range(1, array.shape[0]-1):
        for j in range(1, array.shape[1]-1):
            updated[i,j] = (array[i-1,j] + array[i+1,j] + array[i,j-1] + array[i,j+1]) / 4
            
    return updated

In [ ]:
phi = np.random.randn(64,64)

In [ ]:
%timeit jacobi(phi)

In [ ]:
@numba.jit
def jacobi_numba(array):    
    updated = array.copy()
    
    for i in range(1, array.shape[0]-1):
        for j in range(1, array.shape[1]-1):
            updated[i,j] = (array[i-1,j] + array[i+1,j] + array[i,j-1] + array[i,j+1]) / 4
            
    return updated

In [ ]:
%timeit jacobi_numba(phi)

#### Code generation for `scipy` applications—the case of multiple parameters

We'll perform the parametrized definite integral

$$I(z) = \int_0^\pi \cos(z \sin \theta) d\theta$$

With Numba, we need to use a more specialized decorator than the simple `numba.jit`, and we need to take all parameters as a C array of double-precision floats. The C function takes a first argument of the number of parameters, and a second argument of a pointer to the array.

In [ ]:
# numba will generate a function with C-like calling interface ("cfunc")
# that takes a 32-bit integer and a pointer to an array of 64-bit floats,
# and returns a single float

@numba.cfunc("float64(int32,CPointer(float64))")
def integrand_nc(n, x):
    # unpack the array, Python-style
    theta, z = x[0], x[1]
    
    return math.cos(z * math.sin(theta))

`scipy.integrate.quad` also needs some help to understand what arguments it's being given. That help is provided by `scipy.LowLevelCallable`, which can take several descriptions of C-like functions. Note that we provide the value of additional parameters (here $z$) as a tuple with the keyword argument `args`.

In [ ]:
si.quad(scipy.LowLevelCallable(integrand_nc.ctypes), 0, math.pi, args=(5,))

With Cython, we cannot work entirely within the notebook as we did before, but we need to write a Cython header and  a Cython source file that will be compiled to a C extension that we can import and use. Note the use of the iPython magic `%%file` to write the content of the cell to a file.

In [ ]:
%%file integrand.pxd

# a quasi-C declaration of our Cython function; note "cdef"
cdef double integrand_cc(int n, double[] x)

In [ ]:
%%file integrand.pyx

# "cimport" gives us access to functions in the C standard library
from libc.math cimport sin, cos

# "cdef" creates a function callable from C only
# (while def in the %%cython block above, creates a Python callable)
cdef double integrand_cc(int n, double x[]):
    return cos(x[1] * sin(x[0]))

We can take advantage of the `pyximport` module, which takes care of compiling the extension transparently. Note that C extensions can only be imported once, so modifying the code, recompiling it with the same name, and reimporting it would have no effect. I have wasted lots of debugging code this way!

In [ ]:
import pyximport
pyximport.install(language_level=3)  # Python 3 of course

In [ ]:
import integrand

And again we need some work to dig out and wrap the C-like function we're calling.

In [ ]:
si.quad(scipy.LowLevelCallable(integrand.__pyx_capi__['integrand_cc']), 0, math.pi, args=(5,))

That's it!